In [ ]:
import torch
import torchaudio

print(torch.__version__)
print(torchaudio.__version__)

torch.random.manual_seed(0)
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
print(device)

In [ ]:
import os
MEDIA_DIR = "/Volumes/KINGSTON/veteran_interviews"
# index = 0
# audio_path = f"{MEDIA_DIR}/{index}/audio.mp3"
# video_path = f"{MEDIA_DIR}/{index}/video.mp4"
# # check if the audio file exists
# if not os.path.exists(audio_path):
#     raise FileNotFoundError(f"Audio file not found: {audio_path}")
# # check if the video file exists
# if not os.path.exists(video_path):
#     raise FileNotFoundError(f"Video file not found: {video_path}")

# develop a function to return media type, and audio/ video path based on index
def get_media_type_and_path(index):
    audio_path = f"{MEDIA_DIR}/{index}/audio.mp3"
    video_path = f"{MEDIA_DIR}/{index}/video.mp4"
    if os.path.exists(video_path):
        return "video", video_path
    elif os.path.exists(audio_path):
        return "audio", audio_path
    else:
        return False
    
get_media_type_and_path(0)  # Test the function with index 0

In [ ]:
# load parquet file
import pandas as pd
df = pd.read_parquet("../datasets/veterans_history_project_resources.parquet")
df['media_type'] = df.index.to_series().apply(lambda x: get_media_type_and_path(x)[0] if get_media_type_and_path(x) else None)
df['media_filepath'] = df.index.to_series().apply(lambda x: get_media_type_and_path(x)[1] if get_media_type_and_path(x) else None)

In [ ]:
# create a sample dataset of 100 rows where returned media type is audio
df_sample = df[df['media_type'] == 'audio'].sample(n=10, random_state=42)

In [ ]:
import pprint
# testing_raw_transcript = df_sample['fulltext_file_str'][730]
# strip content from the xml (only get the text between <p> tags)
import re
def strip_xml_tags(text):
    # make sure the input is a string
    if not isinstance(text, str):
        # force it to be a string
        text = str(text)
    # Remove all XML tags except <p> and <speaker>
    # Find the position of the first <speaker> tag
    speaker_match = re.search(r'<speaker>.*?</speaker>', text, re.DOTALL)
    if not speaker_match:
        return ""
    start_pos = speaker_match.end()
    # Only search for <p>...</p> after the first <speaker>
    paragraphs = re.findall(r'<p>(.*?)</p>', text[start_pos:], re.DOTALL)
    # Remove everything after "[Conclusion of Interview]"
    result = []
    for para in paragraphs:
        if "[Conclusion of Interview]" in para:
            break
        result.append(para)
    return '\n'.join(result)
# testing_raw_transcript_stripped = strip_xml_tags(testing_raw_transcript)
# apply this function to the fulltext_file_str column
df_sample['raw_transcript_stripped'] = df_sample['fulltext_file_str'].apply(strip_xml_tags)

In [ ]:
df_sample['raw_transcript_stripped']

In [ ]:
df_sample['media_filepath']

In [ ]:
# save the sample dataset to a new parquet file
df_sample.to_parquet("../datasets/veterans_history_project_sample.parquet", index=False)

In [ ]:
from datasets import load_dataset
# load parquet file as a Hugging Face dataset
hf_dataset = load_dataset('parquet', data_files="../datasets/veterans_history_project_sample.parquet")

In [ ]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")

In [ ]:
import torchaudio

def preprocess_audio(file_path):
    waveform, sample_rate = torchaudio.load(file_path)
    if sample_rate != 16000:
        resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000)
        waveform = resampler(waveform)
    return waveform

def normalize_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)
    return text

def prepare_dataset(batch):
    audio = preprocess_audio(batch["media_filepath"])
    batch["input_values"] = processor(audio, sampling_rate=16000).input_values[0]
    batch["labels"] = processor.tokenizer(batch["raw_transcript_stripped"]).input_ids
    return batch

In [ ]:
dataset = hf_dataset.map(prepare_dataset)

In [ ]:
# save dataset to disk
dataset.save_to_disk("../datasets/veterans_history_project_sample_processed")

In [ ]:
# retrieve the dataset from disk
from datasets import load_from_disk
dataset = load_from_disk("../datasets/veterans_history_project_sample_processed")

In [ ]:
from datasets import DatasetDict

# Split 90% train, 10% validation
split_dataset = dataset['train'].train_test_split(test_size=0.1)

# Rename 'test' split to 'validation'
split_dataset = DatasetDict({
    'train': split_dataset['train'],
    'validation': split_dataset['test']
})

In [ ]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

In [ ]:
from importlib import reload
from transformers import Trainer, TrainingArguments
import accelerate
reload(accelerate)
# reload(TrainingArguments)
# reload(Trainer)

training_args = TrainingArguments(
    output_dir="./checkpoints",
    per_device_train_batch_size=1,
    eval_strategy="no",
    num_train_epochs=2,
    fp16=False,
    save_strategy="no",
    dataloader_num_workers=0,
    logging_steps=10,
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=split_dataset["train"],
    eval_dataset=split_dataset["validation"],
    tokenizer=processor.feature_extractor,
)

In [ ]:
import torch
print(torch.backends.mps.is_available())  # Should be True
print(torch.backends.mps.is_built())       # Should be True

In [ ]:
trainer.train()

END for HF train here^

In [ ]:
torchaudio.set_audio_backend("soundfile")
# Load and resample audio
waveform, sample_rate = torchaudio.load(wav_filepath)

In [ ]:
waveform = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000)(waveform)

In [ ]:
waveform

In [ ]:
from datasets import load_dataset
parentNaId = "653144"
filepath = f'../datasets/{parentNaId}_transcriptions_with_audio.parquet'
filepath = f'../datasets/veterans_history_project_resources.parquet'
# Load dataset from CSV
dataset = load_dataset('parquet', data_files=filepath)

In [ ]:
dataset

In [ ]:
dataset["train"][0]['audio_filepaths'][0]
# create new column to store the filepath after conversion
# audio_filepaths are lists, only take the first element for conversion
dataset = dataset.map(lambda x: {'audio_filepath_1st': [fp[0].replace("./", "../datasets/") for fp in x['audio_filepaths']]}, batched=True)
dataset = dataset.map(lambda x: {'transcription_str': [next(iter(t.values()))['transcription'] for t in x['transcription']]}, batched=True)
dataset = dataset.map(lambda x: {'audio_filepath_1st': [convert_mp3_to_wav(fp) for fp in x['audio_filepath_1st']]}, batched=True)

In [ ]:
dataset["train"][0]['audio_filepath_1st']

In [ ]:
dataset["train"][0]['transcription_str']

In [ ]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer.from_pretrained("facebook/wav2vec2-large-960h")

def preprocess_text(batch):
    batch["input_ids"] = tokenizer(batch["transcription_str"], padding=True, truncation=True).input_ids
    return batch

    # Convert tokenized output to a numpy array to ensure consistent dtype
    tokenized = tokenizer(texts, padding=True, truncation=True, return_tensors='np')
    batch["input_ids"] = tokenized["input_ids"].tolist()
    batch["attention_mask"] = tokenized["attention_mask"].tolist()
    return batch

# Apply preprocessing
dataset = dataset.map(preprocess_text, batched=True)

In [ ]:
from transformers import Wav2Vec2ForCTC, Wav2Vec2Tokenizer

# Load tokenizer and model
tokenizer = Wav2Vec2Tokenizer.from_pretrained("facebook/wav2vec2-large-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-large-960h")

In [ ]:
# Update model configuration for new vocabulary size
model.config.pad_token_id = tokenizer.pad_token_id
model.config.vocab_size = len(tokenizer)

In [ ]:
train_dataset = dataset['train']
eval_dataset = dataset['train']


In [ ]:
from torch.utils.data import DataLoader

# Define a custom collator
def data_collator(batch):
    audio_features = [item["input_values"] for item in batch]
    labels = [item["labels"] for item in batch]
    return {"input_values": audio_features, "labels": labels}

# Create DataLoader
train_loader = DataLoader(
    train_dataset, batch_size=16, shuffle=True, collate_fn=data_collator
)
eval_loader = DataLoader(
    eval_dataset, batch_size=16, shuffle=False, collate_fn=data_collator
)

In [ ]:
for i in train_loader:
    print(i)
    break

In [ ]:
dataset['train']

In [ ]:
import torch
from torch.optim import AdamW

# Define optimizer and scheduler
optimizer = AdamW(model.parameters(), lr=3e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=500, gamma=0.1)

# Training loop
for epoch in range(5):
    model.train()
    for batch in train_loader:
        # Use "input_ids" as both input and label, to avoid KeyError
        input_ids = batch["input_ids"]
        labels = batch["input_ids"]

        # Forward pass
        outputs = model(input_ids, labels=labels)
        loss = outputs.loss

        # Backward pass
        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

    print(f"Epoch {epoch + 1} completed with loss {loss.item():.4f}")